In [4]:
import h5py
import json
import numpy as np
import os
import pandas as pd

img_data = './data/img_data'
text_data = './data/text_data'

In [2]:
# df = pd.read_json(os.path.join(text_data, 'Easy/IR_train_easy.json'))
# df = df.T.sort_index()
# df.dialog = df.dialog.apply(lambda arr: ' '.join([sent for item in arr for sent in item]))
# df['all_words'] = df[['caption', 'dialog']].apply(lambda x: x[0] + ' ' + x[1], axis = 1)
# df = df[['caption', 'dialog', 'all_words', 'img_list', 'target', 'target_img_id']]
# s = df.apply(lambda x: pd.Series(x['img_list']),axis=1).stack().reset_index(level=1, drop=True)
# s.name = 'img_id'
# df = df.drop('img_list', axis=1).join(s).drop('target', axis=1).reset_index(drop= True)
# df['target_onehot'] = np.where(df['target_img_id'] == df['img_id'], 1, 0)
# s = []
# for img_id in df.img_id.iteritems():
#     h5_id = visual_feat_mapping[str(img_id[1])]
#     img_feat = img_features[h5_id]
#     s.append(img_feat)

# df['img_feat'] = pd.Series(s)
# df[['all_words', 'img_feat', 'target_onehot']].to_csv('./data/train_data_easy_bow.csv')

In [3]:
# df_val = pd.read_json(os.path.join(text_data, 'Easy/IR_val_easy.json'))
# df_val = df_val.T.sort_index()
# df_val.dialog = df_val.dialog.apply(lambda arr: ' '.join([sent for item in arr for sent in item]))
# df_val['all_words'] = df_val[['caption', 'dialog']].apply(lambda x: x[0] + ' ' + x[1], axis = 1)
# df_val = df_val[['caption', 'dialog', 'all_words', 'img_list', 'target', 'target_img_id']]
# s = df_val.apply(lambda x: pd.Series(x['img_list']),axis=1).stack().reset_index(level=1, drop=True)
# s.name = 'img_id'
# df_val = df_val.drop('img_list', axis=1).join(s).drop('target', axis=1).reset_index(drop= True)
# df_val['target_onehot'] = np.where(df_val['target_img_id'] == df_val['img_id'], 1, 0)
# s = []
# for img_id in df_val.img_id.iteritems():
#     h5_id = visual_feat_mapping[str(img_id[1])]
#     img_feat = img_features[h5_id]
#     s.append(img_feat)

# df_val['img_feat'] = pd.Series(s)
# df_val[['all_words', 'img_feat', 'target_onehot']].to_csv('./data/val_data_easy_bow.csv')

In [4]:
# df_test = pd.read_json(os.path.join(text_data, 'Easy/IR_test_easy.json'))
# df_test = df_test.T.sort_index()
# df_test.dialog = df_test.dialog.apply(lambda arr: ' '.join([sent for item in arr for sent in item]))
# df_test['all_words'] = df_test[['caption', 'dialog']].apply(lambda x: x[0] + ' ' + x[1], axis = 1)
# df_test = df_test[['caption', 'dialog', 'all_words', 'img_list', 'target', 'target_img_id']]
# s = df_test.apply(lambda x: pd.Series(x['img_list']),axis=1).stack().reset_index(level=1, drop=True)
# s.name = 'img_id'
# df_test = df_test.drop('img_list', axis=1).join(s).drop('target', axis=1).reset_index(drop= True)
# df_test['target_onehot'] = np.where(df_test['target_img_id'] == df_test['img_id'], 1, 0)
# s = []
# for img_id in df_test.img_id.iteritems():
#     h5_id = visual_feat_mapping[str(img_id[1])]
#     img_feat = img_features[h5_id]
#     s.append(img_feat)

# df_test['img_feat'] = pd.Series(s)
# df_test[['all_words', 'img_feat', 'target_onehot']].to_csv('./data/test_data_easy_bowf.csv')

In [31]:
for data in ['train', 'val', 'test']:
    for complexity in ['easy', 'hard']:
        print(data, complexity)
        df = pd.read_json(os.path.join(text_data, '{0}/IR_{1}_{2}.json'\
                                       .format(complexity.capitalize(), data, complexity)))
        df = df.T.sort_index()
        df.dialog = df.dialog\
            .apply(lambda arr: ' \n '.join([sent for item in arr for sent in item]))
        df['all_words'] = df[['caption', 'dialog']]\
            .apply(lambda x: x[0] + ' \n ' + x[1], axis = 1)
        df = df[['caption', 'dialog', 'all_words', 'img_list', 'target', 'target_img_id']]
        s = df.apply(lambda x: pd.Series(x['img_list']),axis=1)\
            .stack()\
            .reset_index(level=1, drop=True)
        s.name = 'img_id'
        df = df.drop('img_list', axis=1).join(s).drop('target', axis=1).reset_index(drop= True)
        df['target_onehot'] = np.where(df['target_img_id'] == df['img_id'], 1, 0)
        
        df[['all_words', 'img_id', 'target_onehot']].to_csv('./data/{0}_data_{1}_bow.csv'\
                                                             .format(data, complexity), 
                                                              index = False)
        

train easy
train hard
val easy
val hard
test easy
test hard


In [29]:
pd.read_csv('./data/train_data_easy_bow.csv').all_words[0]

'a person that is laying next to a dog \n is this a child or adult ? adult \n male or female ? male \n are they inside or outside ? inside \n are they laying on the floor ? yes, but there is a blanket in between them and the floor \n is the floor carpeted or wooden ? it is tile \n what color is the blanket ? red and white \n what color is the tile ? orange red \n what breed is the dog ? boxer \n does the dog look healthy and happy ? yes \n what color is the dog ? tan'

In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

class ImageTextDataSet(Dataset):

    def __init__(self, csv_file):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
        """
        self.imagetext_frame = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.imagetext_frame)

    def __getitem__(self, idx):
        text = self.imagetext_frame.iloc[idx, 0].split(' \n ')
        img_id = self.imagetext_frame.iloc[idx, 1]
        target = self.imagetext_frame.iloc[idx, 2]
        h5_id = visual_feat_mapping[str(img_id)]
        img_feat = img_features[h5_id]
        sample = {'text': text, 'img_features': img_feat, 'target': target}
        return sample

In [2]:
data_train = ImageTextDataSet(csv_file='./data/train_data_easy_bow.csv')
dataloader_train = DataLoader(data_train, batch_size=4,
                        shuffle=True, num_workers=4)

data_val = ImageTextDataSet(csv_file='./data/val_data_easy_bow.csv')
dataloader_val = DataLoader(data_val, num_workers=4)

data_test = ImageTextDataSet(csv_file='./data/test_data_easy_bow.csv')
dataloader_test = DataLoader(data_test, num_workers=4)


In [5]:
# coding: utf-8

"""
CBOW

Based on Graham Neubig's DyNet code examples:
  https://github.com/neubig/nn4nlp2017-code
  http://phontron.com/class/nn4nlp2017/

""" 

from collections import defaultdict
import time
import random
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
UNK = w2i["<unk>"]

def read_dataset(dataloader):
    for batch in dataloader:
        text = batch['text']
        for sentences in text:
            for sentence in sentences:
                resulting_sentence = sentence.lower().strip()
                yield ([w2i[x] for x in resulting_sentence.split(" ")])

# Read in the data
train = list(read_dataset(dataloader_train))
w2i = defaultdict(lambda: UNK, w2i)
val = list(read_dataset(dataloader_val))
test = list(read_dataset(dataloader_test))

nwords = len(w2i)

In [5]:
pd.read_csv('./data/train_data_easy_bow.csv').all_words[0].split('\n')

['a person that is laying next to a dog ',
 ' is this a child or adult ? adult ',
 ' male or female ? male ',
 ' are they inside or outside ? inside ',
 ' are they laying on the floor ? yes, but there is a blanket in between them and the floor ',
 ' is the floor carpeted or wooden ? it is tile ',
 ' what color is the blanket ? red and white ',
 ' what color is the tile ? orange red ',
 ' what breed is the dog ? boxer ',
 ' does the dog look healthy and happy ? yes ',
 ' what color is the dog ? tan']

In [6]:
idx = 0
for idx, batch in enumerate(dataloader_train):
    if idx < 1:
        print(batch)
        idx += 1
    else:
        break

{'text': [('a giraffe eating green leaves in the forest', 'a bunch of boys wearing the same colors supporting their team', 'a orange on a plate on a table', "the top of the heads of a pair of zebra's"), ('how is weather ? sky is white or grayish, overcast', 'how many boys have an you seen ? 7 with visible faces', 'any people there ? no', 'is this in color ? yes'), ('is there any other animals ? no', "how old do they look ? most look in their 20's and 1 is an older guy", 'is the orange peeled ? no', 'is this a close up ? yes'), ('is there any trees ? yes, lots behind in background', 'what ethnicity are they ? white and hispanic from what i can tell', 'what color is the plate ? white', 'can you see any trees ? no'), ('is it in field ? sort of, but grass is short', 'what colors are they wearing ? red and white', 'is it a glass plate ? yes', 'can you see any grass ? no'), ('is there path ? not that i can see, no', "can you tell what sport it is ? not really, but i'd guess basketball", 'is 

In [6]:
import torch.nn.functional as F
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, img_feat_size, output_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim + img_feat_size, output_dim)
        self.selu = nn.SELU()
        self.linear2 = nn.Linear(output_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_text, input_img_feat, batch_size):
        input_text = input_text.view(batch_size, -1)
        x = self.embeddings(input_text)
        x = torch.sum(x, 1)
        x = torch.cat([x, input_img_feat], 1)
        x = self.linear1(x)
        x = self.selu(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x
    

model = CBOW(nwords, 64, 2048, 256).cuda()
print(model)

optimizer = optim.Adam(model.parameters())

CBOW (
  (embeddings): Embedding(28356, 64)
  (linear1): Linear (2112 -> 256)
  (selu): SELU
  (linear2): Linear (256 -> 1)
  (sigmoid): Sigmoid ()
)


In [ ]:
def preprocess_text(text):
        max_len = 0
        resulting_text = []
        for sentences in text:
            resulting_sentences = []
            for sentence in sentences:
                resulting_sentence = sentence.lower().strip().split(" ")
                max_len = max(max_len, len(resulting_sentence))
                resulting_sentences.append([w2i[x] for x in resulting_sentence])
            resulting_text.append(resulting_sentences)
        resulting_text = [[sentence + ([UNK] * (max_len - len(sentence)))
                          for sentence in sentences] for sentences in resulting_text]
        return torch.LongTensor(resulting_text)
    
def train(epoch, dataloader, batch_size = 4, cuda = False):
    model.train()
    train_loss = 0.0
    correct = 0.0
    for batch_idx, sample in enumerate(dataloader):
        text_data = preprocess_text(sample['text'])
        img_feat = sample['img_features'] 
        target = sample['target'].float()
        if cuda:
            text_data, img_feat, target = text_data.cuda(), img_feat.cuda(), target.cuda()
        text_data, img_feat, target = Variable(text_data), Variable(img_feat), Variable(target)
        optimizer.zero_grad()
        output = model(text_data, img_feat, batch_size)
        loss = F.binary_cross_entropy(output, target, size_average=False)
        loss.backward()
        optimizer.step()
        train_loss += loss.data[0] # sum up batch loss
        pred = output >= 0.5
        pred = pred.view(-1) == target.byte()
        correct += pred.float().cpu().sum().int().data[0]
    train_loss /= len(dataloader.dataset)
    print('\nTrain epoch: {} Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        epoch, train_loss, correct, len(dataloader.dataset),
        100.0 * correct / len(dataloader.dataset)))

def test(dataloader, batch_size = 1, cuda = False):
    model.eval()
    test_loss = 0
    correct = 0
    for batch_idx, sample in enumerate(dataloader_test):
        text_data = preprocess_text(sample['text'])
        img_feat = sample['img_features'] 
        target = sample['target'].float()
        if cuda:
            text_data, img_feat, target = text_data.cuda(), img_feat.cuda(), target.cuda()
        text_data, img_feat, target = Variable(text_data, volatile=True), Variable(img_feat, volatile=True), Variable(target)
        output = model(text_data, img_feat, batch_size)
        loss = F.binary_cross_entropy(output, target, size_average=False)
        test_loss += loss.data[0] # sum up batch loss
        pred = output >= 0.5
        pred = pred.eq(target.byte())
        correct += pred.cpu().sum()

    test_loss /= len(dataloader_test.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(dataloader.dataset),
        100. * correct.float() / len(dataloader.dataset)))

import shutil
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

for epoch in range(1, 100):
    train(epoch, dataloader_train, cuda = True)
    prec1 = test(dataloader_val, cuda = True)
    is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)
    save_checkpoint({
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)


Train epoch: 1 Average loss: 0.0001, Accuracy: 3.0/400000 (0%)



In [ ]:
def evaluate(model, data):
    """Evaluate a model on a data set."""
    correct = 0.0
    
    for words, tag in data:
        lookup_tensor = Variable(torch.LongTensor([words]))
        scores = model(lookup_tensor)
        predict = scores.data.numpy().argmax(axis=1)[0]

        if predict == tag:
            correct += 1

    return correct, len(data), correct/len(data)


optimizer = optim.SGD(model.parameters(), lr=0.01)

for ITER in range(100):

    random.shuffle(train)
    train_loss = 0.0
    start = time.time()

    for words, tag in train:

        # forward pass
        lookup_tensor = Variable(torch.LongTensor([words]))
        scores = model(lookup_tensor)
        loss = nn.CrossEntropyLoss()
        target = Variable(torch.LongTensor([tag]))
        output = loss(scores, target)
        train_loss += output.data[0]

        # backward pass
        model.zero_grad()
        output.backward()

        # update weights
        optimizer.step()

    print("iter %r: train loss/sent=%.4f, time=%.2fs" % 
          (ITER, train_loss/len(train), time.time()-start))

    # evaluate
    _, _, acc = evaluate(model, dev)
    print("iter %r: test acc=%.4f" % (ITER, acc))
